# Intake
Author: [Jason Boutte](https://github.com/jasonb5)

Date: 08/17/23

Last Updated: 08/17/23

Sources:
- https://github.com/intake/intake
- https://github.com/esgf-nimbus/nimbus/tree/main/dockerfiles/minimal-notebook/intake_es

Documentation:
- https://intake.readthedocs.io/en/latest/

## Changelog

## Overview
In this notebook we'll use the `intake_es` package to search and access local data on the Nimbus cluster.

The `intake_es` package provides a new Catalog that can query an ElasticSearch (ES) instance that we've populated with the metadata of locally available data. The package also includes everything required to access these ES indexes.

The following projects have been indexed:
- CMIP6
- CMIP5

**Warning:** The `intake_es` package is *beta* and can change.

In [1]:
import intake
import json

import warnings

warnings.simplefilter('ignore')

## 1. Look at the available indexes
The local catalogs are available under `intake.cat.local`.

In [2]:
list(intake.cat.local)

['cmip6', 'cmip5']

## 2. Describe the index
Here we'll use the CMIP6 catalog and start looking at the index.

This description provides information about the index, e.g. settings, column names and types.

In [3]:
cmip6 = intake.cat.local.cmip6

cmip6, cmip6.describe_index

2023-08-17 23:51:48,561 [INFO]: _transport.py(perform_request:336) >> GET https://jupyterhub-es-http:9200/cmip6 [status:200 duration:0.008s]


(<cmip6 catalog with 0 datasets>,
 {'cmip6': {'aliases': {},
   'mappings': {'properties': {'activity_drs': {'type': 'keyword'},
     'experiment_id': {'type': 'keyword'},
     'grid_label': {'type': 'keyword'},
     'institution_id': {'type': 'keyword'},
     'member_id': {'type': 'keyword'},
     'mip_era': {'type': 'text',
      'fields': {'keyword': {'type': 'keyword', 'ignore_above': 256}}},
     'path': {'type': 'keyword'},
     'source_id': {'type': 'keyword'},
     'table_id': {'type': 'keyword'},
     'variable_id': {'type': 'keyword'},
     'version': {'type': 'keyword'}}},
   'settings': {'index': {'routing': {'allocation': {'include': {'_tier_preference': 'data_content'}}},
     'number_of_shards': '1',
     'provided_name': 'cmip6',
     'creation_date': '1691823236137',
     'number_of_replicas': '1',
     'uuid': 'ZFX2Tyl9QPCYckqYGMkdUw',
     'version': {'created': '8090099'}}}}})

## 3. Display the total number of datasets indexed

In [4]:
cmip6.count

2023-08-17 23:51:48,572 [INFO]: _transport.py(perform_request:336) >> POST https://jupyterhub-es-http:9200/cmip6/_count [status:200 duration:0.002s]


6020752

## 4. List the index fields
This will list the searchable fields of the index.

In [5]:
cmip6.fields

2023-08-17 23:51:48,585 [INFO]: _transport.py(perform_request:336) >> GET https://jupyterhub-es-http:9200/cmip6 [status:200 duration:0.001s]


{'activity_drs': 'keyword',
 'experiment_id': 'keyword',
 'grid_label': 'keyword',
 'institution_id': 'keyword',
 'member_id': 'keyword',
 'mip_era': 'keyword',
 'path': 'keyword',
 'source_id': 'keyword',
 'table_id': 'keyword',
 'variable_id': 'keyword',
 'version': 'keyword'}

## 5. List the top values of a field
This will list the top (10) values of a field along with the number of matches.

In [6]:
cmip6.field_top('activity_drs')

2023-08-17 23:51:48,602 [INFO]: _transport.py(perform_request:336) >> POST https://jupyterhub-es-http:9200/cmip6/_search [status:200 duration:0.002s]


{'PAMIP': 2084111,
 'DCPP': 2013403,
 'ScenarioMIP': 622666,
 'CMIP': 461601,
 'DAMIP': 351248,
 'AerChemMIP': 108898,
 'VolMIP': 65776,
 'RFMIP': 54305,
 'C4MIP': 48118,
 'LUMIP': 48112}

## 6. List the all the values of a field
This will list all the values of a field along with the number of matches.

*This operation can take a long time if there are many unique values in a field*

In [7]:
cmip6.field_nunique('activity_drs')

2023-08-17 23:51:48,617 [INFO]: _transport.py(perform_request:336) >> POST https://jupyterhub-es-http:9200/cmip6/_search [status:200 duration:0.002s]
2023-08-17 23:51:48,619 [INFO]: _transport.py(perform_request:336) >> POST https://jupyterhub-es-http:9200/cmip6/_search [status:200 duration:0.002s]
2023-08-17 23:51:48,621 [INFO]: _transport.py(perform_request:336) >> POST https://jupyterhub-es-http:9200/cmip6/_search [status:200 duration:0.002s]
2023-08-17 23:51:48,624 [INFO]: _transport.py(perform_request:336) >> POST https://jupyterhub-es-http:9200/cmip6/_search [status:200 duration:0.002s]


{'AerChemMIP': 108898,
 'C4MIP': 48118,
 'CDRMIP': 11011,
 'CFMIP': 33157,
 'CMIP': 461601,
 'DAMIP': 351248,
 'DCPP': 2013403,
 'FAFMIP': 19000,
 'GMMIP': 9870,
 'GeoMIP': 17797,
 'HighResMIP': 32588,
 'ISMIP6': 1585,
 'LS3MIP': 12931,
 'LUMIP': 48112,
 'OMIP': 11141,
 'PAMIP': 2084111,
 'PMIP': 13433,
 'RFMIP': 54305,
 'ScenarioMIP': 622666,
 'VolMIP': 65776,
 'test': 1}

## 7. Search for some data
Calling `search` will apply the query and retrun a new Catalog containing the results. With the new catalog you can explore the results or continue to refine your search.

**WARNING:** By default a search will return all results, this can take a long time if there are a large number of matches.

#### Notes:
- It's recommended to use `field_top` and `field_nunique` methods to find some starting values to search for.
- Passing `dry_run=True` will return just the number of matches for a query.
- Passing a list to a field will return results matching either value, e.g. `variable_id=['tas', 'pr']` will return matches with either `tas` or `pr`.
- You can negate a value by prefixing it with `!`, e.g. `variable_id='!tas'` will return all variable_id's except `tas`.

In [8]:
result = cmip6.search(
    activity_drs='CMIP',
    institution_id=['MIROC', 'NCAR'],
    source_id=['MIROC6', 'CESM2'],
    member_id=[f'r{x+1}i1p1f1' for x in range(4)],
    experiment_id='historical',
    table_id=['!3hr', '!day'],
    variable_id='tas'
)
result

2023-08-17 23:51:48,635 [INFO]: _transport.py(perform_request:336) >> POST https://jupyterhub-es-http:9200/cmip6/_search [status:200 duration:0.004s]
2023-08-17 23:51:48,639 [INFO]: _transport.py(perform_request:336) >> POST https://jupyterhub-es-http:9200/cmip6/_search [status:200 duration:0.003s]


,mip_era,activity_drs,institution_id,source_id,experiment_id,member_id,table_id,variable_id,grid_label,version,path
0,CMIP6,CMIP,MIROC,MIROC6,historical,r2i1p1f1,Amon,tas,gn,v20181212,/p/css03/esgf_publish/CMIP6/CMIP/MIROC/MIROC6/...
1,CMIP6,CMIP,MIROC,MIROC6,historical,r1i1p1f1,Amon,tas,gn,v20181212,/p/css03/esgf_publish/CMIP6/CMIP/MIROC/MIROC6/...
2,CMIP6,CMIP,MIROC,MIROC6,historical,r4i1p1f1,Amon,tas,gn,v20181212,/p/css03/esgf_publish/CMIP6/CMIP/MIROC/MIROC6/...
3,CMIP6,CMIP,MIROC,MIROC6,historical,r3i1p1f1,Amon,tas,gn,v20181212,/p/css03/esgf_publish/CMIP6/CMIP/MIROC/MIROC6/...
4,CMIP6,CMIP,NCAR,CESM2,historical,r2i1p1f1,Amon,tas,gn,v20190308,/p/css03/esgf_publish/CMIP6/CMIP/NCAR/CESM2/hi...
5,CMIP6,CMIP,NCAR,CESM2,historical,r1i1p1f1,Amon,tas,gn,v20190308,/p/css03/esgf_publish/CMIP6/CMIP/NCAR/CESM2/hi...
6,CMIP6,CMIP,NCAR,CESM2,historical,r4i1p1f1,Amon,tas,gn,v20190308,/p/css03/esgf_publish/CMIP6/CMIP/NCAR/CESM2/hi...
7,CMIP6,CMIP,NCAR,CESM2,historical,r3i1p1f1,Amon,tas,gn,v20190308,/p/css03/esgf_publish/CMIP6/CMIP/NCAR/CESM2/hi...


## 8. List the unique keys of the results
These keys can be used to open the datasets.

In [9]:
list(result)

['CMIP6.CMIP.MIROC.MIROC6.historical.r2i1p1f1.Amon.tas.gn.v20181212',
 'CMIP6.CMIP.MIROC.MIROC6.historical.r1i1p1f1.Amon.tas.gn.v20181212',
 'CMIP6.CMIP.MIROC.MIROC6.historical.r4i1p1f1.Amon.tas.gn.v20181212',
 'CMIP6.CMIP.MIROC.MIROC6.historical.r3i1p1f1.Amon.tas.gn.v20181212',
 'CMIP6.CMIP.NCAR.CESM2.historical.r2i1p1f1.Amon.tas.gn.v20190308',
 'CMIP6.CMIP.NCAR.CESM2.historical.r1i1p1f1.Amon.tas.gn.v20190308',
 'CMIP6.CMIP.NCAR.CESM2.historical.r4i1p1f1.Amon.tas.gn.v20190308',
 'CMIP6.CMIP.NCAR.CESM2.historical.r3i1p1f1.Amon.tas.gn.v20190308']

## 9. Open a dataset using a key
There are a few different methods to open the data at this point.
- `read()` will load all the data into memory.
- `read_chunked()` or `to_dask()` will open the data lazily, only reading the metadata.
- `to_xcdat()` will open the data using `xcdat`.

In [10]:
ds = result['CMIP6.CMIP.MIROC.MIROC6.historical.r2i1p1f1.Amon.tas.gn.v20181212'].to_dask()
ds

<xarray.Dataset>
Dimensions:    (time: 1980, bnds: 2, lat: 128, lon: 256)
Coordinates:
  * time       (time) datetime64[ns] 1850-01-16T12:00:00 ... 2014-12-16T12:00:00
  * lat        (lat) float64 -88.93 -87.54 -86.14 -84.74 ... 86.14 87.54 88.93
  * lon        (lon) float64 0.0 1.406 2.812 4.219 ... 354.4 355.8 357.2 358.6
    height     float64 2.0
Dimensions without coordinates: bnds
Data variables:
    time_bnds  (time, bnds) datetime64[ns] dask.array<chunksize=(1200, 2), meta=np.ndarray>
    lat_bnds   (time, lat, bnds) float64 dask.array<chunksize=(1200, 128, 2), meta=np.ndarray>
    lon_bnds   (time, lon, bnds) float64 dask.array<chunksize=(1200, 256, 2), meta=np.ndarray>
    tas        (time, lat, lon) float32 dask.array<chunksize=(1200, 128, 256), meta=np.ndarray>
Attributes: (12/44)
    Conventions:            CF-1.7 CMIP-6.2
    activity_id:            CMIP
    branch_method:          standard
    branch_time_in_child:   0.0
    branch_time_in_parent:  10958.0
    creation_date:          2018-11-30T17:09:54Z
    ...                     ...
    title:                  MIROC6 output prepared for CMIP6
    variable_id:            tas
    variant_label:          r2i1p1f1
    license:                CMIP6 model data produced by MIROC is licensed un...
    cmor_version:           3.3.2
    tracking_id:            hdl:21.14100/3428dd30-c17d-4356-82b9-d92bda1289b5

## 10. Open all datasets and store in a dictionary
**WARNING:** For results containg a large number of datasets this can take a long time.

By default this will open all datasets using the `to_dask()` method. Alternatively you can pass `use_xcdat=True` and the datasets will be opened using `to_xcdat()` and `**kwargs` will be passed through.

In [11]:
datasets = result.to_dataset_dict(use_xcdat=True, chunks={"time": 20})
list(datasets.values())[0]

<xarray.Dataset>
Dimensions:    (time: 1980, bnds: 2, lat: 128, lon: 256)
Coordinates:
  * time       (time) datetime64[ns] 1850-01-16T12:00:00 ... 2014-12-16T12:00:00
  * lat        (lat) float64 -88.93 -87.54 -86.14 -84.74 ... 86.14 87.54 88.93
  * lon        (lon) float64 0.0 1.406 2.812 4.219 ... 354.4 355.8 357.2 358.6
    height     float64 2.0
Dimensions without coordinates: bnds
Data variables:
    time_bnds  (time, bnds) datetime64[ns] dask.array<chunksize=(1200, 2), meta=np.ndarray>
    lat_bnds   (time, lat, bnds) float64 dask.array<chunksize=(1200, 128, 2), meta=np.ndarray>
    lon_bnds   (time, lon, bnds) float64 dask.array<chunksize=(1200, 256, 2), meta=np.ndarray>
    tas        (time, lat, lon) float32 dask.array<chunksize=(1200, 128, 256), meta=np.ndarray>
Attributes: (12/44)
    Conventions:            CF-1.7 CMIP-6.2
    activity_id:            CMIP
    branch_method:          standard
    branch_time_in_child:   0.0
    branch_time_in_parent:  10958.0
    creation_date:          2018-11-30T17:09:54Z
    ...                     ...
    title:                  MIROC6 output prepared for CMIP6
    variable_id:            tas
    variant_label:          r2i1p1f1
    license:                CMIP6 model data produced by MIROC is licensed un...
    cmor_version:           3.3.2
    tracking_id:            hdl:21.14100/3428dd30-c17d-4356-82b9-d92bda1289b5

## 11. Open all datasets and store in a DataTree
**WARNING:** For results containg a large number of datasets this can take a long time.

In [12]:
dt = result.to_datatree()
dt['CMIP6/CMIP']

DataTree('CMIP', parent="CMIP6")
├── DataTree('MIROC')
│   └── DataTree('MIROC6')
│       └── DataTree('historical')
│           ├── DataTree('r2i1p1f1')
│           │   └── DataTree('Amon')
│           │       └── DataTree('tas')
│           │           └── DataTree('gn')
│           │               └── DataTree('v20181212')
│           │                       Dimensions:    (time: 1980, bnds: 2, lat: 128, lon: 256)
│           │                       Coordinates:
│           │                         * time       (time) datetime64[ns] 1850-01-16T12:00:00 ... 2014-12-16T12:00:00
│           │                         * lat        (lat) float64 -88.93 -87.54 -86.14 -84.74 ... 86.14 87.54 88.93
│           │                         * lon        (lon) float64 0.0 1.406 2.812 4.219 ... 354.4 355.8 357.2 358.6
│           │                           height     float64 2.0
│           │                       Dimensions without coordinates: bnds
│           │                       Data variables:
│           │                           time_bnds  (time, bnds) datetime64[ns] dask.array<chunksize=(1200, 2), meta=np.ndarray>
│           │                           lat_bnds   (time, lat, bnds) float64 dask.array<chunksize=(1200, 128, 2), meta=np.ndarray>
│           │                           lon_bnds   (time, lon, bnds) float64 dask.array<chunksize=(1200, 256, 2), meta=np.ndarray>
│           │                           tas        (time, lat, lon) float32 dask.array<chunksize=(1200, 128, 256), meta=np.ndarray>
│           │                       Attributes: (12/44)
│           │                           Conventions:            CF-1.7 CMIP-6.2
│           │                           activity_id:            CMIP
│           │                           branch_method:          standard
│           │                           branch_time_in_child:   0.0
│           │                           branch_time_in_parent:  10958.0
│           │                           creation_date:          2018-11-30T17:09:54Z
│           │                           ...                     ...
│           │                           title:                  MIROC6 output prepared for CMIP6
│           │                           variable_id:            tas
│           │                           variant_label:          r2i1p1f1
│           │                           license:                CMIP6 model data produced by MIROC is licensed un...
│           │                           cmor_version:           3.3.2
│           │                           tracking_id:            hdl:21.14100/3428dd30-c17d-4356-82b9-d92bda1289b5
│           ├── DataTree('r1i1p1f1')
│           │   └── DataTree('Amon')
│           │       └── DataTree('tas')
│           │           └── DataTree('gn')
│           │               └── DataTree('v20181212')
│           │                       Dimensions:    (time: 1980, bnds: 2, lat: 128, lon: 256)
│           │                       Coordinates:
│           │                         * time       (time) datetime64[ns] 1850-01-16T12:00:00 ... 2014-12-16T12:00:00
│           │                         * lat        (lat) float64 -88.93 -87.54 -86.14 -84.74 ... 86.14 87.54 88.93
│           │                         * lon        (lon) float64 0.0 1.406 2.812 4.219 ... 354.4 355.8 357.2 358.6
│           │                           height     float64 2.0
│           │                       Dimensions without coordinates: bnds
│           │                       Data variables:
│           │                           time_bnds  (time, bnds) datetime64[ns] dask.array<chunksize=(1200, 2), meta=np.ndarray>
│           │                           lat_bnds   (time, lat, bnds) float64 dask.array<chunksize=(1200, 128, 2), meta=np.ndarray>
│           │                           lon_bnds   (time, lon, bnds) float64 dask.array<chunksize=(1200, 256, 2), meta=np.ndarray>
│           │                           tas        (time, lat, lon) float32 dask.arra